In [1]:
from datetime import datetime
from cropclassification.image_processing import hls_organizer
from cropclassification.image_processing import fmask_processer
from cropclassification.image_processing import clouds_remover
from cropclassification.image_processing import cdl_reclassifier
from cropclassification.temporal_composites import date_interval_selector
from cropclassification.temporal_composites import cv_calculator
from cropclassification.temporal_composites import percentiles_calculator
from cropclassification.temporal_composites import spectral_indices_calculator

from importlib import reload

## 1. Initial preprocessing
This code will read the Harmonized Landsat Sentinel-2 images (HLS) and perform the following steps:
- Organize HLS bands
- Create a cloud mask band
- Remove the clouds from the HLS bands

### 1.1 Run the function to organize the HLS bands
This function will organize the HLS images by converting the julian days in datatime, and correcting the projection to UTM.

In [5]:
# Organize images names, dates, UTM projection and export to folder pre-process
# Define the input directory to locate the HLS images
hls_raw_images = r'Z:\dbcenter\images\hls\scenes'
# Define the output directory to save the organized images
base_dir = r'C:\crop_classification'
# Modify this string according to the HLS tile images
tile_dir = '15SYT'
# Define the period of interest
start_date = '20220301'
end_date = '20221130'
# Run the function to organize the HLS bands
hls_organizer.organize_hls(hls_raw_images, base_dir, tile_dir, start_date, end_date)


## 1.2 Run the function to create a cloud mask
This function will decode the HLS Fmask to get the cirrus (bit 0), clouds (bit 1), adjacent clouds (bit 2), and cloud shadow (bit 3) to create a cloud mask.

In [6]:
# Decodes the cloud and cloud shadow information, generates a binary cloud mask, and safe the cloud band in the folder pre-process
# Define the base directory to locate the HLS organized images and export the cloud mask
base_dir = r'C:\crop_classification'
# Call the function with the desired output directory
fmask_processer.process_fmask(base_dir)

C:\crop_classification\pre_process\hls_organized\20220301\20220301_S30_T15SYT_Fmask.tif
C:\crop_classification\pre_process\hls_organized\20220305\20220305_L30_T15SYT_Fmask.tif
C:\crop_classification\pre_process\hls_organized\20220306\20220306_S30_T15SYT_Fmask.tif
C:\crop_classification\pre_process\hls_organized\20220311\20220311_S30_T15SYT_Fmask.tif
C:\crop_classification\pre_process\hls_organized\20220316\20220316_S30_T15SYT_Fmask.tif
C:\crop_classification\pre_process\hls_organized\20220321\20220321_L30_T15SYT_Fmask.tif
C:\crop_classification\pre_process\hls_organized\20220321\20220321_S30_T15SYT_Fmask.tif
C:\crop_classification\pre_process\hls_organized\20220326\20220326_S30_T15SYT_Fmask.tif
C:\crop_classification\pre_process\hls_organized\20220329\20220329_L30_T15SYT_Fmask.tif
C:\crop_classification\pre_process\hls_organized\20220331\20220331_S30_T15SYT_Fmask.tif
C:\crop_classification\pre_process\hls_organized\20220405\20220405_S30_T15SYT_Fmask.tif
C:\crop_classification\pre_proce

## 1.3 Apply the function to remove clouds from the HLS images
This function will apply the cloud mask to all HLS bands, apply HLS scaler factor (10000), and export the HLS images to pre-process folder

In [2]:
# Apply the cloud mask to all HLS bands, apply HLS scaler factor (10000), and export the HLS images to pre-process folder
# The code will process only the following bands, if you want to add more bands, modify the dictionary in the code "clouds_remover":
# 'S30': {2: 'B02', 3: 'B03', 4: 'B04', 8: 'B08', 11: 'B11', 12: 'B12'},
# 'L30': {2: 'B02', 3: 'B03', 4: 'B04', 5: 'B05', 6: 'B06', 7: 'B07'}

# Define the base directory to locate the HLS organized images, and the cloud mask
base_dir = r'C:\crop_classification'

# Call the function to process HLS images
clouds_remover.process_hls_images(base_dir)

## 2. Time composites and Vegetation indices
This part creates a quarterly temporal composite of bands:
- Coefficient of variation and average bands
- Percentages of 10%, 25%, 50%, 75% and 90%
- Spectral indices

### 2.1 Calculate the percentiles and the coefficient of variation
This functions will create a quarterly temporal composition and calculate the percentiles of 10%, 25%, 50%, 75% and 90%, and the coefficient of variation.

In [3]:
# Functions to calculate the percentages and coefficient of variation
base_dir = r'C:\crop_classification'
# Specify the interest bands
bands = ['B02', 'B03', 'B04', 'NIR', 'SWR1', 'SWR2']
# Define a custom interval of months if needed.
custom_intervals = [(3, 5), (6, 8), (9, 11)]
# Function generates a date interval to bi-monthly, quarterly, semester, annual and custom. Default is bi-monthly
date_ranges = date_interval_selector.get_band_dates(bands, interval_type='custom', year=2022, custom_intervals=custom_intervals)
# Call the function to calculate the percentiles of 10%, 25%, 50%, 75%, and 90% of the images
percentiles_calculator.calculate_percentiles_hls(base_dir, date_ranges, bands)
# Call the function to calculate the variation coefficient and average images
cv_calculator.calculate_hls_cv(base_dir, date_ranges, bands)


C:\crop_classification\pre_process\hls_cloudless\20220301\20220301_S30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220305\20220305_L30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220306\20220306_S30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220311\20220311_S30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220316\20220316_S30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220321\20220321_L30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220321\20220321_S30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220326\20220326_S30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220329\20220329_L30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220331\20220331_S30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220405\20220405_S30_T15SYT_B02.tif
C:\crop_classification\pre_process\hls_cloudless\20220

### 2.2 Calculate the spectral indices
This function will calculate the following spectral indices for each percentile:
- Enhanced Vegetation Index (EVI)
- Normalized Burn Ratio (NBR)
- Normalized Difference Built-up Index (NDBI)
- Normalized Difference Tillage Index (NDTI)
- Normalized Difference Water Index (NDWI)

In [4]:
# Calculate the spectral indices from the percentile images
# Base directory to locate the HLS percentiles and storage the spectral indices in the folder 'inputdata'
base_dir = r'C:\crop_classification'
# Run the function to calculate the spectral indices
spectral_indices_calculator.spectral_indices(base_dir)

C:\crop_classification\temporal_composites\percentiles\03_B02_p10.tif
C:\crop_classification\temporal_composites\percentiles\03_B02_p25.tif
C:\crop_classification\temporal_composites\percentiles\03_B02_p50.tif
C:\crop_classification\temporal_composites\percentiles\03_B02_p75.tif
C:\crop_classification\temporal_composites\percentiles\03_B02_p90.tif
C:\crop_classification\temporal_composites\percentiles\06_B02_p10.tif
C:\crop_classification\temporal_composites\percentiles\06_B02_p25.tif
C:\crop_classification\temporal_composites\percentiles\06_B02_p50.tif
C:\crop_classification\temporal_composites\percentiles\06_B02_p75.tif
C:\crop_classification\temporal_composites\percentiles\06_B02_p90.tif
C:\crop_classification\temporal_composites\percentiles\09_B02_p10.tif
C:\crop_classification\temporal_composites\percentiles\09_B02_p25.tif
C:\crop_classification\temporal_composites\percentiles\09_B02_p50.tif
C:\crop_classification\temporal_composites\percentiles\09_B02_p75.tif
C:\crop_classificati

## Reclassified CDL images
Reclasfied the CDL images to the following classes:
- corn
- cotton
- rice
- sorghum
- soybeans
- wheat
- other

In [2]:
# Define the CLD directory with UTM projection and the dimension of the HLS images
cld_raw_image = r'C:\crop_classification\pre_process\cropscape\cdl_crops_2022.tif'
# Define the directory to storage the reclassified CDL
cld_reclassified = r'C:\crop_classification\pre_process\cropscape\cdl_reclassified.tif'
# Run the function to reclassify the CDL image
cdl_reclassifier.reclassify_raster(cld_raw_image, cld_reclassified)